# Coursework 1

In [1]:
import numpy as np
import math
import random

Returns a grid of random numbers of size width x height

In [2]:
def generate_grid(width, height):
    grid = np.random.randint(9, size=(height, width))
    
    return grid

Returns a unique identifier for grid cell

In [3]:
def id_from_cell(grid, x, y):
    width = len(grid[0])
    
    return x + (width * y)

Returns a cell's position from it's id

In [4]:
def cell_from_id(grid, id):
    width = len(grid[0])
    x = id % width
    y = (id - x) // width
    
    return (x, y)

Returns the id of the final cell in a grid (bottom right)

In [5]:
def get_end_id(grid):
    width = len(grid[0])
    height = len(grid)

    return id_from_cell(grid, width - 1, height - 1)

Returns the value of cell at position (x, y) or at cell id

In [6]:
def get_val_at_pos(grid, x, y):
    val = grid[y][x]

    return val

def get_val_at_id(grid, id):
    pos = cell_from_id(grid, id)
    val = get_val_at_pos(grid, pos[0], pos[1])

    return val

Returns a dictionary of a cell's details

In [7]:
def gen_cell_dict(grid, id):
    width = len(grid[0])
    height = len(grid)
    n = id - width
    e = id + 1
    s = id + width
    w = id - 1

    # sets neighbour to -1 if out of grid bounds
    if n < 0:
        n = -1
    if id // width != e // width:
        e = -1
    if s >= width * height:
        s = -1
    if id // width != w // width:
        w = -1
    
    # populates the dictionary for a cell
    cell_dict = {
        'visited':False,
        'parent':-1,
        'north':n,
        'east':e,
        'south':s,
        'west':w
    }

    return cell_dict

Returns a list of the ids of the unvisited neighbours of a cell

In [8]:
def get_cell_unvisited_neighbours(grid_dict, id):
    cell_dict = grid_dict[id]

    neighbour_ids = []
    for direction in ['north', 'east', 'south', 'west']:
        direction_id = cell_dict[direction]
        
        # only adds if neighbour is within grid, and has yet to be visited
        if direction_id != -1 and not grid_dict[direction_id]['visited']:
            neighbour_ids.append(cell_dict[direction])
    
    return neighbour_ids

Returns a list of the ids of the neighbours of a cell

In [9]:
def get_cell_neighbours(grid_dict, id):
    cell_dict = grid_dict[id]

    neighbour_ids = []
    for direction in ['north', 'east', 'south', 'west']:
        direction_id = cell_dict[direction]

        # only adds if cell is within grid
        if direction_id != -1:
            neighbour_ids.append(cell_dict[direction])
    
    return neighbour_ids

Adds a key value pair to a given dictionary

In [10]:
def add_to_dict(given_dict, key, value):
    given_dict[key] = value

Returns a dictionary of all cells, containing their neighbours

In [11]:
def gen_grid_dict(grid):
    width = len(grid[0])
    height = len(grid)

    dicts = {}
    for y in range(height):
        for x in range(width):
            id = id_from_cell(grid, x, y)
            cell_dict = gen_cell_dict(grid, id)
            dicts[id] = cell_dict
    
    return dicts

Makes a movement by setting the new cell's parent to the previous cell's id

In [12]:
def move_cell(grid_dict, parent_cell_id, cell_id):
    cell_dict = grid_dict[cell_id]
    cell_dict['parent'] = parent_cell_id

Returns the length of the generated path

In [13]:
def path_len(grid, path_ids):
    value = 0
    for id in path_ids:
        value += get_val_at_id(grid, id)
    
    return value

Returns the path from start to finish

In [14]:
def get_path(grid, grid_dict):
    cell_id = get_end_id(grid)

    cells = []
    last_run = False
    while(cell_id != -1 or last_run):
        cells.append(cell_id)
        cell_id = grid_dict[cell_id]['parent']

        if cell_id == -1 and not last_run:
            last_run = True
            
        if cell_id == -1 and last_run:
            last_run = False
    
    cells.reverse()
    return cells

Zigzag Algorithm:

Zig zags east then south from the start of the list to the end

In [15]:
def zigzag(grid, grid_dict):
    end_id = get_end_id(grid)
    
    cell_id = 0
    while cell_id != end_id:
        cell_dict = grid_dict[cell_id]

        east_id = cell_dict['east']
        # prevents travel east when at right edge of grid
        if not east_id == -1:
            move_cell(grid_dict, cell_id, east_id)
            cell_id = east_id

        cell_dict = grid_dict[cell_id]

        south_id = cell_dict['south']
        # prevents travel south when at bottom edge of grid
        if not south_id == -1:
            move_cell(grid_dict, cell_id, south_id)
            cell_id = south_id
    
    return get_path(grid, grid_dict)

Dijkstra's Algorithm:

In [16]:
def dijkstras(grid, grid_dict):
    end_id = get_end_id(grid)

    unvisited_ids = []

    for key in grid_dict:
        unvisited_ids.append(key)
        add_to_dict(grid_dict[key], 'dist_val', None)

    # sets distance value of start cell to its length
    grid_dict[0]['dist_val'] = get_val_at_id(grid, 0)
    id = unvisited_ids[0]

    while id != end_id:
        neighbours = get_cell_unvisited_neighbours(grid_dict, id)
        cell_dict = grid_dict[id]
        for neighbour_id in neighbours:
            distance = cell_dict['dist_val'] + get_val_at_id(grid, neighbour_id)
            neighbour_dict = grid_dict[neighbour_id]

            # updates distance if neighbour has not been accessed before
            if neighbour_dict['dist_val'] is None:
                move_cell(grid_dict, id, neighbour_id)
                neighbour_dict['dist_val'] = distance
            
            # updates distance if neighbours previous distance was longer
            elif neighbour_dict['dist_val'] > distance:
                move_cell(grid_dict, id, neighbour_id)
                neighbour_dict['dist_val'] = distance
        
        # marks cell as visited and removes it from the unvisited list
        cell_dict['visited'] = True
        unvisited_ids.remove(id)

        smallest_dist = None
        next_id = -1

        # chooses which cell to consider next
        for potential_next_id in unvisited_ids:
            next_cell_dict = grid_dict[potential_next_id]
            next_dist = next_cell_dict['dist_val']

            # always chooses the given cell if none has been chosen already
            if smallest_dist is None:
                next_id = potential_next_id
                smallest_dist = next_dist

            # chooses cell if it has already been considered by the main algorithm, and has a smaller distance than the previouslychosen cell
            elif next_dist is not None and next_dist < smallest_dist:
                next_id = potential_next_id
                smallest_dist = next_dist
        
        id = next_id

    return get_path(grid, grid_dict)

Ant Colony Optimisation Algorithm:

In [17]:
def aco(grid, grid_dict):

    # adds pheremone characteristics to all cell dictionaries
    for key in grid_dict:
        add_to_dict(grid_dict[key], 'pheremone_level', ACO_INITIAL_PHEREMONE)
        add_to_dict(grid_dict[key], 'pheremone_update', 0)
    
    last_path = []
    # runs through chosen number of iterations
    for i in range(ACO_ITERATIONS):
        path = generate_solutions(grid, grid_dict)
        last_path = path

    set_parents(grid_dict, last_path)
    return(last_path)

Generates solutions for a chosen number of ants and returns the shortest path chosen

In [18]:
def generate_solutions(grid, grid_dict):
    end_id = get_end_id(grid)

    paths = []
    # simulates a given number of ants
    for ant in range(ACO_ANT_COUNT):
        current_id = 0
        visited_ids = [0]

        # ant chooses which cell to visit next until it reaches the end
        while current_id != end_id:
            current_id = edge_selection(grid, grid_dict, current_id)
            visited_ids.append(current_id)
        
        paths.append(visited_ids)

        # marks all cells as unvisited
        for key in grid_dict:
            grid_dict[key]['visited'] = False
        # adds pheremone to visited cells
        add_pheremone(grid, grid_dict, visited_ids)
    
    # generates alist of all the path lengths generated by the ants
    path_lens = []
    for path in paths:
        path_lens.append(path_len(grid, path))
    
    # finds the shortest path
    min_path_index = path_lens.index(min(path_lens))
    min_path = paths[min_path_index]

    # adds extra pheremone to the shortest distance
    add_pheremone(grid, grid_dict, min_path)
    update_pheremone(grid_dict)

    return min_path

Determines the next edge an ant will move to

In [19]:
def edge_selection(grid, grid_dict, id):
    neighbours = get_cell_neighbours(grid_dict, id)
    neighbour_desirabilities = []

    # calculates the desirability of a cell's neighbour
    for neighbour in neighbours:
        # gives a very low desirability if cell has been visited before
        # mostly eliminates chances of cycles, but without giving the ant no movement options
        visited_before = ACO_VISITED_INFLUENCE if grid_dict[neighbour]['visited'] else 1

        # adds 1 to move cost to avoid dividing by 0
        move_cost = get_val_at_id(grid, neighbour) + 1

        # modifies the desirability of pheremones and path length by their influence factors
        pheremone_level = grid_dict[neighbour]['pheremone_level'] ** ACO_PHEREMONE_INFLUENCE
        move_desirability = (1 / move_cost) ** ACO_DISTANCE_INFLUENCE

        # the overall desirability of a cell
        move_value = pheremone_level * move_desirability * visited_before
        neighbour_desirabilities.append(move_value)

    probabilities = []

    # calculates probability of moving to a given neighbour
    desirability_sum = sum(neighbour_desirabilities)
    for i in range(len(neighbours)):
        probability = neighbour_desirabilities[i] / desirability_sum
        probabilities.append(probability)
    
    grid_dict[id]['visited'] = True

    # choosen the neighbour to move to
    edge_choice = random.choices(neighbours, weights= probabilities)

    return edge_choice[0]

Determines the amount of pheremone to add to a visited cell

In [20]:
def add_pheremone(grid, grid_dict, visited_ids):
    # chooses amount of pheremone to add based on journey length (more for shorter journey, less for longer)
    journey_length = path_len(grid, visited_ids)
    added_pheremone = ACO_EXPECTED_DISTANCE / journey_length

    # adds pheremone to all visited cells
    for id in visited_ids:
        grid_dict[id]['pheremone_update'] = added_pheremone

Updates the pheremone values for every cell

In [21]:
def update_pheremone(grid_dict):
    for key in grid_dict:
        # gets the prior level of pheremone and the amount to add
        old_level = grid_dict[key]['pheremone_level']
        added_pher = grid_dict[key]['pheremone_update']

        # calculates the new pheremone level and resets the amount to add
        grid_dict[key]['pheremone_level'] = (1 - ACO_EVAPORATION_COEFFICIENT) * old_level + added_pher
        grid_dict[key]['pheremone_update'] = 0

Updates the parents of all the cells in the path for the purposes of evaluation using the predefined functions

In [22]:
def set_parents(grid_dict, path_ids):

    for i in range(len(path_ids) - 1):
        move_cell(grid_dict, path_ids[i], path_ids[i+1])
    move_cell(grid_dict, -1, 0)

Higher order function for running an algorithm, catches exceptions if function doesn't work

In [23]:
# runs algorithm if asked to generate a grid
def run_algorithm(func, x_size, y_size, show_grid=True, show_path_as_ids=False):
    grid = generate_grid(x_size, y_size)

    run_algorithm(func, grid, show_grid, show_path_as_ids)

# runs algorithm if a grid is provided
def run_algorithm(func, grid, show_grid=True, show_path_as_ids=False):
    grid_dict = gen_grid_dict(grid)
    try:
        path = func(grid, grid_dict)
        # path = get_path(grid, grid_dict)

        if(show_grid):
            print(grid)
        show_visited(grid, path)

        # chooses wheter to show path as cell ids or grid coordinates
        if(show_path_as_ids):
            print(path)
        else:
            path_coords = []
            for id in path:
                path_coords.append(cell_from_id(grid, id))
            print(path_coords)

        print('{0} algorithm gives a path length of {1}'.format(func.__name__, path_len(grid, path)))
    except:
        print("Invalid function provided/function throws an error")

Runs algorithm for testing purposes

In [24]:
# runs algorithm if asked to generate a grid
def run_algorithm_unsafe(func, x_size, y_size):
    grid = generate_grid(x_size, y_size)

    run_algorithm_unsafe(func, grid)

# runs algorithm if a grid is provided
def run_algorithm_unsafe(func, grid):
    grid_dict = gen_grid_dict(grid)
    path = func(grid, grid_dict)
    # path = get_path(grid, grid_dict)

    print(grid)
    print(path)

    show_visited(grid, path)

    print(path_len(grid, path))

Times how long an algorithm takes to execute

In [25]:
from timeit import default_timer as timer

def time_algorithm(func, x_size, y_size, show_grid= False, show_path_as_ids= False, safe= False):
    start = timer()
    if safe:
        run_algorithm(func, x_size, y_size, show_grid, show_path_as_ids)
    else:
        run_algorithm_unsafe(func, x_size, y_size)
    end = timer()
    print(func.__name__ + " algorithm executed in " + str(end - start) + " seconds")

def time_algorithm(func, grid, show_grid=False, show_path_as_ids= False, safe=False):
    start = timer()
    if safe:
        run_algorithm(func, grid, show_grid, show_path_as_ids)
    else:
        run_algorithm_unsafe(func, grid)
    end = timer()
    print(func.__name__ + " algorithm executed in " + str(end - start) + " seconds")

In [26]:
def show_visited(grid, path):
    width = len(grid[0])
    height = len(grid)

    zero_grid = np.zeros((height, width), dtype=int)

    for id in path:
        (x, y) = cell_from_id(grid, id)
        zero_grid[y][x] += 1
    
    print(zero_grid)

In [27]:
# constants effecting ACO algorithm behaviour
ACO_ITERATIONS = 25
ACO_ANT_COUNT = 20
ACO_INITIAL_PHEREMONE = 1
ACO_PHEREMONE_INFLUENCE = 1.3
ACO_DISTANCE_INFLUENCE = 1.0
ACO_VISITED_INFLUENCE = 0.05
ACO_EXPECTED_DISTANCE = 50
ACO_EVAPORATION_COEFFICIENT = 0.5

testing_grid = generate_grid(10, 10)
print(testing_grid)

time_algorithm(aco, testing_grid, safe=True)
time_algorithm(zigzag, testing_grid, safe=True)
time_algorithm(dijkstras, testing_grid, safe=True)

[[4 0 2 3 0 3 1 0 0 3]
 [0 7 1 7 0 2 1 4 1 4]
 [7 1 3 7 6 1 1 5 4 4]
 [4 6 6 5 4 8 6 2 2 3]
 [5 5 4 4 0 1 1 2 6 4]
 [4 8 6 3 5 0 2 5 5 4]
 [1 8 7 6 1 5 1 8 5 8]
 [8 6 8 0 4 3 4 6 0 0]
 [2 8 7 5 5 0 4 0 0 3]
 [1 6 0 8 8 8 2 8 3 4]]
[[1 1 1 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0]
 [0 1 1 0 0 0 0 0 0 0]
 [0 1 1 1 1 0 0 0 0 0]
 [0 0 0 0 1 1 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 1 1 1 1 0]
 [0 0 0 0 0 0 0 0 1 1]]
[(0, 0), (1, 0), (2, 0), (2, 1), (2, 2), (1, 2), (1, 3), (2, 3), (3, 3), (4, 3), (4, 4), (5, 4), (5, 5), (5, 6), (5, 7), (5, 8), (6, 8), (7, 8), (8, 8), (8, 9), (9, 9)]
aco algorithm gives a path length of 52
aco algorithm executed in 1.7091654999999264 seconds
[[1 1 0 0 0 0 0 0 0 0]
 [0 1 1 0 0 0 0 0 0 0]
 [0 0 1 1 0 0 0 0 0 0]
 [0 0 0 1 1 0 0 0 0 0]
 [0 0 0 0 1 1 0 0 0 0]
 [0 0 0 0 0 1 1 0 0 0]
 [0 0 0 0 0 0 1 1 0 0]
 [0 0 0 0 0 0 0 1 1 0]
 [0 0 0 0 0 0 0 0 1 1]
 [0 0 0 0 0 0 0 0 0 1]]
[(0, 0), (1, 0), (1, 1), (2, 1), (2, 2), (3, 